# Add to the path

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

# Import Libraries

In [2]:
from Tensorized_Layers.TRL import TRL
from Utils.Num_parameter import count_parameters

import torch
import torch.nn as nn
from torch import optim
import time

# Dummy Data

In [3]:
device = 'cuda'
batch_size = 5
dummy = torch.rand(batch_size, 14, 14, 16, 16, 3).to(device)
print(f'Current shape is : {dummy.shape}')

Current shape is : torch.Size([5, 14, 14, 16, 16, 3])


# TRL
- input size is (5, 14, 14, 16, 16, 3)
- output size is (16,16,3) # excluding the ignored modes
- ignored modes are 0: batch , 1:patch index 1, 2: patch index 2
- ranks are (16,16,3) : for the input modes that are not ignored and (16,16,3) for the output size
- device is set to cuda

In [4]:
trl = TRL(input_size=dummy.shape,
            output=(16,16,3),
            rank=(16,16,3,16,16,3),
            ignore_modes=(0,1,2),
            device = device
            ).to(device)
st = time.time()
output = trl(dummy)
elapsed = time.time() - st
print(f'output shape of trl: {output.shape}')

print(f'This trl has {count_parameters(trl)} parameters')
print(f'This trl took : {elapsed}')

output shape of trl: torch.Size([5, 14, 14, 16, 16, 3])
This trl has 591634 parameters
This trl took : 0.025475740432739258


In [5]:
for key in trl.state_dict():
    print(key, trl.state_dict()[key].device)

b cuda:0
core cuda:0
u0 cuda:0
u1 cuda:0
u2 cuda:0
u3 cuda:0
u4 cuda:0
u5 cuda:0


In [6]:
new_classifier = nn.Sequential(
    trl,
    nn.Flatten(),
    nn.Linear(150528,2)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_classifier.parameters())

temp_y = torch.randint(0, 2, (dummy.shape[0],)).to(device)

optimizer.zero_grad()    
outputs = new_classifier(dummy)
loss = criterion(outputs, temp_y)
loss.backward()
for p in trl.parameters():
    print(p.shape, p.device, p.grad.device)
optimizer.step()

print('second backward')
optimizer.zero_grad()    
outputs = new_classifier(dummy)
loss = criterion(outputs, temp_y)
loss.backward()
for p in trl.parameters():
    print(p.shape, p.device, p.grad.device)
optimizer.step()

torch.Size([16, 16, 3]) cuda:0 cuda:0
torch.Size([16, 16, 3, 16, 16, 3]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([3, 3]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([3, 3]) cuda:0 cuda:0
second backward
torch.Size([16, 16, 3]) cuda:0 cuda:0
torch.Size([16, 16, 3, 16, 16, 3]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([3, 3]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([16, 16]) cuda:0 cuda:0
torch.Size([3, 3]) cuda:0 cuda:0


# Compare to tltorch TRL from Tensorly

In [7]:
device = 'cuda'
batch_size = 5
dummy = torch.rand(batch_size, 16, 16, 3).to(device)
print(f'Current shape is : {dummy.shape}')

Current shape is : torch.Size([5, 16, 16, 3])


## Forward Pass

In [8]:
trl1 = TRL(input_size=dummy.shape,
            output=(16,16,3),
            rank=(16,16,3,16,16,3),
            ignore_modes=(0,),
            device = device
            ).to(device)
st = time.time()
output1 = trl1(dummy)
elapsed = time.time() - st
print(f'output shape of trl (our method): {output1.shape}')

print(f'This trl (our method) has {count_parameters(trl1)} parameters')
print(f'This trl (our method) took : {elapsed}')

output shape of trl (our method): torch.Size([5, 16, 16, 3])
This trl (our method) has 591634 parameters
This trl (our method) took : 0.00179290771484375


In [9]:
import tltorch

trl2 = tltorch.TRL(input_shape = (16,16,3), 
                   output_shape = (16,16,3),
                   bias = True,
                   factorization='tucker',
                   rank = (16,16,3,16,16,3)).to(device)
st = time.time()
output2 = trl2(dummy)
elapsed = time.time() - st
print(f'output shape of trl (tltorch method): {output2.shape}')

print(f'This trl (tltorch method) has {count_parameters(trl2)} parameters')
print(f'This trl (tltorch method) took : {elapsed}')

output shape of trl (tltorch method): torch.Size([5, 16, 16, 3])
This trl (tltorch method) has 591634 parameters
This trl (tltorch method) took : 0.0005576610565185547


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Backward Pass

In [10]:
new_classifier = nn.Sequential(
    trl1,
    nn.Flatten(),
    nn.Linear(16*16*3,2)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_classifier.parameters())

temp_y = torch.randint(0, 2, (dummy.shape[0],)).to(device)

st = time.time()
optimizer.zero_grad()    
outputs = new_classifier(dummy)
loss = criterion(outputs, temp_y)
loss.backward()
optimizer.step()
elapsed = time.time() - st
print(f'This backward trl (our method) took : {elapsed}')


This backward trl (our method) took : 0.001783609390258789


In [11]:
new_classifier = nn.Sequential(
    trl2,
    nn.Flatten(),
    nn.Linear(16*16*3,2)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_classifier.parameters())

temp_y = torch.randint(0, 2, (dummy.shape[0],)).to(device)

st = time.time()
optimizer.zero_grad()    
outputs = new_classifier(dummy)
loss = criterion(outputs, temp_y)
loss.backward()
optimizer.step()
elapsed = time.time() - st
print(f'This backward trl (tltorch method) took : {elapsed}')


This backward trl (tltorch method) took : 0.0019812583923339844
